In [36]:
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go

df_position = pd.read_json("data/raw/ais_position.json")
df_static = pd.read_json("data/raw/ais_static.json")
df_types = pd.read_csv("data/raw/ais_ShipTypes.csv", sep=";")
df_status = pd.read_csv("data/raw/ais_NavigationStatus.csv", sep=";")

df_static['timestamp'] = pd.to_datetime(df_static['timestamp'].str.replace('UTC', ''), format='ISO8601')
df_position.sort_values('MMSI')

,timestamp,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,UserID
25,2025-07-07 12:05:48.711344268 +0000 UTC,205133000,Z26 AVANTI,48.791910,-4.017993,139.7,7,0,11.0,0,205133000
58,2025-07-07 12:06:48.626506775 +0000 UTC,205133000,Z26 AVANTI,48.789972,-4.014490,137.4,7,0,10.7,0,205133000
88,2025-07-07 12:07:49.139222568 +0000 UTC,205133000,Z26 AVANTI,48.787980,-4.011028,132.3,7,0,11.5,0,205133000
69,2025-07-07 12:07:19.383696345 +0000 UTC,205133000,Z26 AVANTI,48.788960,-4.012693,129.3,7,0,11.1,0,205133000
111,2025-07-07 12:08:49.196689771 +0000 UTC,205133000,Z26 AVANTI,48.785983,-4.007473,132.1,7,0,11.2,0,205133000
...,...,...,...,...,...,...,...,...,...,...,...
106,2025-07-07 12:08:42.652530503 +0000 UTC,374496000,ROLLS I,48.250710,-4.973102,78.6,0,0,10.5,0,374496000
79,2025-07-07 12:07:42.659924416 +0000 UTC,374496000,ROLLS I,48.250073,-4.977350,77.8,0,0,10.5,0,374496000
48,2025-07-07 12:06:34.050962435 +0000 UTC,374496000,ROLLS I,48.249367,-4.982165,78.6,0,0,10.5,0,374496000
118,2025-07-07 12:09:04.219820049 +0000 UTC,374496000,ROLLS I,48.250937,-4.971562,77.0,0,0,10.5,0,374496000


In [37]:
# Keep LATEST MMSI static data
df_static_latest = df_static.sort_values("timestamp").drop_duplicates("MMSI", keep="last")
# Add the Ship Type string
df_static_latest = pd.merge(df_static_latest, df_types, on='Type', how='left')
# Calculate Lenght and Width
df_static_latest["Length"] = df_static_latest["Dimension"].apply(lambda x: x.get("A", 0) + x.get("B", 0))
df_static_latest["Width"] = df_static_latest["Dimension"].apply(lambda x: x.get("C", 0) + x.get("D", 0))

# Join Navigation Status to position
df_position_final = pd.merge(df_position, df_status, on='NavigationalStatus', how='left')

# final_df
final_df = pd.merge(df_position_final, df_static_latest, on='MMSI', how='left', suffixes = ('_position', '_static'))
final_df

# QUESTION --> EST CE QUE JE GARDE TOUTES LES POISTIONS ? SI OUI AJOUTER UN POSITIONSTATUS : 1 = Latest, 0 = Old

,timestamp_position,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,...,timestamp_static,Destination,Dimension,Eta,Type,Type Description,Category,SubCategory,Length,Width
0,2025-07-07 12:04:52.625957289 +0000 UTC,228247000,F/V MARVIN,48.718725,-3.965838,84.4,7,-128,0.0,0,...,2025-07-07 12:07:09.476845531+00:00,EN PECHE VHF 9,"{'A': 8, 'B': 7, 'C': 4, 'D': 2}","{'Day': 2, 'Hour': 12, 'Minute': 0, 'Month': 7}",30.0,Fishing,Fishing,NaN,15.0,6.0
1,2025-07-07 12:04:53.680243777 +0000 UTC,227580520,LOUARN,48.380008,-4.497828,119.4,0,-128,0.0,0,...,2025-07-07 12:07:49.134247165+00:00,,"{'A': 10, 'B': 20, 'C': 4, 'D': 4}","{'Day': 0, 'Hour': 0, 'Minute': 0, 'Month': 0}",60.0,"Passenger, all ships of this type",Passenger,all ships of this type,30.0,8.0
2,2025-07-07 12:04:53.827958843 +0000 UTC,227916000,F/V TARZH AN DEIZ,48.718570,-3.965790,96.3,7,127,0.0,0,...,2025-07-07 12:09:18.948623912+00:00,Z0NE DE PECHE,"{'A': 7, 'B': 7, 'C': 3, 'D': 3}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",30.0,Fishing,Fishing,NaN,14.0,6.0
3,2025-07-07 12:04:54.055855315 +0000 UTC,227006680,MISTRAL 10,48.381185,-4.484505,50.0,0,-128,0.0,0,...,2025-07-07 12:05:33.225240261+00:00,BREST,"{'A': 16, 'B': 14, 'C': 9, 'D': 1}","{'Day': 13, 'Hour': 3, 'Minute': 0, 'Month': 5}",52.0,Tug,Tug,NaN,30.0,10.0
4,2025-07-07 12:04:54.210465679 +0000 UTC,227999999,FRENCH WARSHIP,48.369167,-4.514283,89.9,0,0,0.0,0,...,2025-07-07 12:06:56.688257218+00:00,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 7, 'Hour': 7, 'Minute': 7, 'Month': 7}",35.0,Military ops,Military,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2025-07-07 12:09:45.099723401 +0000 UTC,227006680,MISTRAL 10,48.381190,-4.484512,50.0,0,-128,0.0,0,...,2025-07-07 12:05:33.225240261+00:00,BREST,"{'A': 16, 'B': 14, 'C': 9, 'D': 1}","{'Day': 13, 'Hour': 3, 'Minute': 0, 'Month': 5}",52.0,Tug,Tug,NaN,30.0,10.0
142,2025-07-07 12:09:49.142493115 +0000 UTC,205133000,Z26 AVANTI,48.783957,-4.003918,130.2,7,0,11.6,0,...,2025-07-07 12:07:20.619024839+00:00,FISHINGGROUNDS,"{'A': 28, 'B': 10, 'C': 6, 'D': 2}","{'Day': 0, 'Hour': 0, 'Minute': 0, 'Month': 0}",30.0,Fishing,Fishing,NaN,38.0,8.0
143,2025-07-07 12:09:53.23052812 +0000 UTC,228247000,F/V MARVIN,48.718728,-3.965828,92.3,7,-128,0.2,0,...,2025-07-07 12:07:09.476845531+00:00,EN PECHE VHF 9,"{'A': 8, 'B': 7, 'C': 4, 'D': 2}","{'Day': 2, 'Hour': 12, 'Minute': 0, 'Month': 7}",30.0,Fishing,Fishing,NaN,15.0,6.0
144,2025-07-07 12:09:53.727549739 +0000 UTC,227580520,LOUARN,48.379982,-4.497790,114.5,0,-128,0.0,0,...,2025-07-07 12:07:49.134247165+00:00,,"{'A': 10, 'B': 20, 'C': 4, 'D': 4}","{'Day': 0, 'Hour': 0, 'Minute': 0, 'Month': 0}",60.0,"Passenger, all ships of this type",Passenger,all ships of this type,30.0,8.0


In [67]:
final_df["custom_hover"] = (
    final_df["MMSI"].astype(str) + " - " + final_df["ShipName"] + "<br>" +
    "Category : " + final_df["Category"] + "<br>" +
    "Status : " + final_df["Status Description"] + "<br>" +
    "Destination : " + final_df["Destination"] + "<br>" +
    "Position at : " + final_df["timestamp_position"].astype(str)
)


fig = px.scatter_map(final_df, lat="lat", lon="lon", color="Status Description",
                     color_continuous_scale=px.colors.cyclical.IceFire,
                     hover_name="custom_hover"
                     )


for key, group in final_df.groupby(['MMSI', 'UserID']):
    # Sort group if needed for order of lines
    group_sorted = group.sort_values(['timestamp_position']) # or by time if you have it
    print(group_sorted)

    fig.add_trace(go.Scattermap(
        lat=group_sorted['lat'],
        lon=group_sorted['lon'],
        mode='lines',
        hoverinfo = 'text',
        line=dict(color="grey", width=1),
        hovertemplate=(
        "<b>%{customdata[0]} - %{customdata[1]}</b><br>" +
        "Category: %{customdata[2]}<br>" +
        "Status: %{customdata[3]}<br>" +
        "Destination: %{customdata[4]}<br>" +
        "Position at: %{customdata[5]}<br><extra></extra>"
        ),
        customdata= group_sorted[[
            'MMSI', 'ShipName', 'Category', 'Status Description', 'Destination', 'timestamp_position'
            ]].values,
    showlegend=False  # or True if you want legends for lines
    ))

fig.show()

                          timestamp_position       MMSI    ShipName  \
25   2025-07-07 12:05:48.711344268 +0000 UTC  205133000  Z26 AVANTI   
58   2025-07-07 12:06:48.626506775 +0000 UTC  205133000  Z26 AVANTI   
69   2025-07-07 12:07:19.383696345 +0000 UTC  205133000  Z26 AVANTI   
88   2025-07-07 12:07:49.139222568 +0000 UTC  205133000  Z26 AVANTI   
111  2025-07-07 12:08:49.196689771 +0000 UTC  205133000  Z26 AVANTI   
121  2025-07-07 12:09:08.428436351 +0000 UTC  205133000  Z26 AVANTI   
142  2025-07-07 12:09:49.142493115 +0000 UTC  205133000  Z26 AVANTI   

           lat       lon    COG  NavigationalStatus  RateOfTurn   SOG  Spare  \
25   48.791910 -4.017993  139.7                   7           0  11.0      0   
58   48.789972 -4.014490  137.4                   7           0  10.7      0   
69   48.788960 -4.012693  129.3                   7           0  11.1      0   
88   48.787980 -4.011028  132.3                   7           0  11.5      0   
111  48.785983 -4.007473  132.1

In [ ]:
px.line_map()